# Food Manufacture II

Both this model and Food Manufacture I are examples of blending problems. In blending optimization problems, multiple raw materials are combined in a way the meets the stated constraints for the lowest cost. These problems are common in numerous industries including the oil industry (blending different types of crude oil at a refinery) and agriculture (manufacturing feed that meets the different nutritional requirements of different animals).

In this particular example, we will model and solve a production planning problem where we must create a final product from a number of ingredients — each of which has different costs, restrictions, and features. The aim is to create an optimal multi-period production plan that maximizes profit. More details can be found on the Problem Description and Model Formulation tabs below.

Compared to Food Manufacturing I, this example has additional constraints that change the problem type from linear programming (LP) to mixed-integer programming (MIP), making it harder to solve. You can see these additional constraints at the end of the Problem Description section below.

More information on this type of model can be found in the fifth edition of Model Building in Mathematical Programming, by H. Paul Williams.

---
## Problem Description

A manufacturer needs to refine several raw oils and blend them together to produce a given food product that can be sold. The raw oils needed can be divided into two categories:


| Category        | Oil         |
| ------------- |-------------| 
| Vegetable oils:|  VEG 1<br>VEG 2 | 
| Non-vegetable oils:     | OIL 1<br>OIL 2<br>OIL 3      |


The manufacturer can choose to buy raw oils for the current month and/or buy them on the futures market for delivery in a subsequent month. Prices for immediate delivery and in the futures market are given below in USD/ton:

| Month |	VEG 1 |	VEG 2 |	OIL 1 |	OIL 2 |	OIL 3|
| ------------- |-------------| -------------| -------------| -------------| -------------| 
| January| 110 |	120 |	130 |	110 |	115|
| February |130 |	130 |	110 |	90|	115|
| March |110 |	140 |	130 |	100 |	95|
| April |120 |	110 |	120 |	120 |	125|
| May | 100 |	120 |	150 |	110 |	105|
| June |  	90 |	100 |	140 |	80| 	135 |

There are a number of additional factors that must be taken into account. These include:

1. The final food product sells for \$150 per ton.
2. Each category of oil (vegetable and non-vegetable) needs to be refined on a different production line.
3. There is limited refinement capacity such that in any given month a maximum of 200 tons of vegetable oils and 250 tons of non-vegetable oils can be refined.
4. Also, there is no waste in the refinement process, so the sum of the raw oils refined will equal the amount of refined oils available.
5. The cost of refining the oils may be ignored.

In addition to the refining limits above, there are limits to the amount of raw oils that can be stored for future use, and there is a cost associated with each ton of oil stored. The limit is 1,000 tons of each raw oil and the storage cost is \$5 per ton per month. The manufacturer cannot store the produced food product or the refined oils.

The final food product must have a hardness between three and six on a given hardness scale. For the purposes of the model, hardness blends linearly and the hardness of each raw oil is:

|Oils |	Hardness|
| ------------- |-------------| 
|VEG 1 |	8.8|
|VEG 2 |	6.1|
|OIL 1 |	2.0|
|OIL2 |	4.2|
|OIL 3| 	5.0|

At the start of January, there are 500 tons of each type of raw oil in storage. For the purpose of the model, this should also be the level of raw oils in storage at the end of June.

This version of the Food Manufacture problem adds the following additional constraints to the first version:

- If an oil is used during a month, the minimum quantity used must be 20 tons.
- The maximum number of oils used in a month is three.
- The use of VEG1 or VEG2 in a given month requires the use of OIL3 in that same month.


Given the above information, what monthly buying and manufacturing decisions should be made in order to maximize profit?

This problem is based on a larger model built for the margarine producer Van den Bergs and Jurgens and discussed in Williams and Redwood (1974).

---
## Model Formulation

### Sets and Indices

$t \in \text{Months}=\{\text{Jan},\text{Feb},\text{Mar},\text{Apr},\text{May},\text{Jun}\}$: Set of months.

$V=\{\text{VEG1},\text{VEG2}\}$: Set of vegetable oils.

$N=\{\text{OIL1},\text{OIL2},\text{OIL3}\}$: Set of non-vegetable oils.

$o \in \text{Oils} = V \cup N$: Set of oils.

### Parameters

$\text{price} \in \mathbb{R}^+$: Sale price of the final product.

$\text{init_store} \in \mathbb{R}^+$: Initial storage amount in tons.

$\text{target_store} \in \mathbb{R}^+$: Target storage amount in tons.

$\text{holding_cost} \in \mathbb{R}^+$: Monthly cost (in USD/ton/month) of keeping in inventory a ton of oil.

$\text{min_consume} \in \mathbb{R}^+$: Minimum number of tons to consume of a given oil in a month.

$\text{veg_cap} \in \mathbb{R}^+$: Installed capacity (in tons) to refine vegetable oils.

$\text{oil_cap} \in \mathbb{R}^+$: Installed capacity (in tons) to refine non-vegetable oils.

$\text{min_hardness} \in \mathbb{R}^+$: lowest hardness allowed for the final product.

$\text{max_hardness} \in \mathbb{R}^+$: highest hardness allowed for the final product.

$\text{hardness}_o \in \mathbb{R}^+$: Hardness of oil $o$.

$\text{max_ingredients} \in \mathbb{N}$: Maximum number of oil types to consume in a given month.

$\text{cost}_{t,o} \in \mathbb{R}^+$: Estimated purchase price for oil $o$ at month $t$.


### Decision Variables

$\text{produce}_t \in \mathbb{R}^+$: Tons of food to produce at month $t$.

$\text{buy}_{t,o} \in \mathbb{R}^+$: Tons of oil $o$ to buy at month $t$.

$\text{consume}_{t,o} \in \mathbb{R}^+$: Tons of oil $o$ to use at month $t$.

$\text{store}_{t,o} \in \mathbb{R}^+$: Tons of oil $o$ to store at month $t$.

$\text{use}_{t,o} \in \{0,1\}$: 1 if oil $o$ is used on month $t$, 0 otherwise. 


### Objective Function

- **Profit**: Maximize the total profit (in USD) of the planning horizon.

\begin{equation}
\text{Maximize} \quad Z = \sum_{t \in \text{Months}}\text{price}*\text{produce}_t - \sum_{t \in \text{Months}}\sum_{o \in \text{Oils}}(\text{cost}_{t,o}*\text{consume}_{t,o} + \text{holding_cost}*\text{store}_{t,o})
\tag{0}
\end{equation}

### Constraints

- **Initial Balance:** The Tons of oil $o$ purchased in January and the ones previously stored should be equal to the Tons of said oil consumed and stored in that month.

\begin{equation}
\text{init store} + \text{buy}_{Jan,o} = \text{consume}_{Jan,o} + \text{store}_{Jan,o} \quad \forall o \in \text{Oils}
\tag{1}
\end{equation}

- **Balance:** The Tons of oil $o$ purchased in month $t$ and the ones previously stored should be equal to the Tons of said oil consumed and stored in that month.

\begin{equation}
\text{store}_{t-1,o} + \text{buy}_{t,o} = \text{consume}_{t,o} + \text{store}_{t,o} \quad \forall (t,o) \in \text{Months} \setminus \{\text{Jan}\} \times \text{Oils}
\tag{2}
\end{equation}

- **Inventory Target**: The Tons of oil $o$ kept in inventory at the end of the planning horizon should hit the target.

\begin{equation}
\text{store}_{Jun,o} = \text{target_store} \quad \forall o \in \text{Oils}
\tag{3}
\end{equation}

- **Refinement Capacity**: Total Tons of oil $o$ consumed in month $t$ cannot exceed the refinement capacity.

\begin{equation}
\sum_{o \in V}\text{consume}_{t,o} \leq \text{veg_cap} \quad \forall t \in \text{Months}
\tag{4.1}
\end{equation}

\begin{equation}
\sum_{o \in N}\text{consume}_{t,o} \leq \text{oil_cap} \quad \forall t \in \text{Months}
\tag{4.2}
\end{equation}

- **Hardness**: The hardness value of the food produced in month $t$ should be within tolerances.

\begin{equation}
\text{min_hardness}*\text{produce}_t \leq \sum_{o \in \text{Oils}} \text{hardness}_o*\text{consume}_{t,o} \leq \text{max_hardness}*\text{produce}_t \quad \forall t \in \text{Months}
\tag{5}
\end{equation}

- **Mass Conservation**: Total Tons of oil consumed in month $t$ should be equal to the Tons of the food produced in that month.

\begin{equation}
\sum_{o \in \text{Oils}}\text{consume}_{t,o} = \text{produce}_t \quad \forall t \in \text{Months}
\tag{6}
\end{equation}

- **Consumption Range**: Oil $o$ can be consumed in month $t$ if we decide to use it in that month, and the Tons consumed should be between 20 and the refinement capacity for its type. 

\begin{equation}
\text{min_consume}*\text{use}_{t,o} \leq \text{consume}_{t,o} \leq \text{veg_cap}*\text{use}_{t,o} \quad \forall (t,o) \in V \times \text{Months}
\tag{7.1}
\end{equation}

\begin{equation}
\text{min_consume}*\text{use}_{t,o} \leq \text{consume}_{t,o} \leq \text{oil_cap}*\text{use}_{t,o} \quad \forall (t,o) \in N \times \text{Months}
\tag{7.2}
\end{equation}

- **Recipe**: The maximum number of oils used in month $t$ must be three.

\begin{equation}
\sum_{o \in \text{Oils}}\text{use}_{t,o} \leq \text{max_ingredients} \quad \forall t \in \text{Months}
\tag{8}
\end{equation}

- **If-then Constraint**: If oils VEG1 or VEG2 are used in month $t$, then OIL3 must be used in that month.

\begin{equation}
\text{use}_{t,\text{VEG1}} \leq \text{use}_{t,\text{OIL3}} \quad \forall t \in \text{Months}
\tag{9.1}
\end{equation}

\begin{equation}
\text{use}_{t,\text{VEG2}} \leq \text{use}_{t,\text{OIL3}} \quad \forall t \in \text{Months}
\tag{9.2}
\end{equation}

---
## Python Implementation
First, we import the Gurobi Python Module and initialize the data structures with the given data.


In [1]:
import numpy as np
import pandas as pd
from gurobipy import *

# tested with Python 3.7 & Gurobi 8

months = ["Jan", "Feb", "Mar", "Apr", "May", "Jun"]

oils = ["VEG1", "VEG2", "OIL1", "OIL2", "OIL3"]

cost = {
    ('Jan', 'VEG1'): 110,
    ('Jan', 'VEG2'): 120,
    ('Jan', 'OIL1'): 130,
    ('Jan', 'OIL2'): 110,
    ('Jan', 'OIL3'): 115,
    ('Feb', 'VEG1'): 130,
    ('Feb', 'VEG2'): 130,
    ('Feb', 'OIL1'): 110,
    ('Feb', 'OIL2'): 90,
    ('Feb', 'OIL3'): 115,
    ('Mar', 'VEG1'): 110,
    ('Mar', 'VEG2'): 140,
    ('Mar', 'OIL1'): 130,
    ('Mar', 'OIL2'): 100,
    ('Mar', 'OIL3'): 95,
    ('Apr', 'VEG1'): 120,
    ('Apr', 'VEG2'): 110,
    ('Apr', 'OIL1'): 120,
    ('Apr', 'OIL2'): 120,
    ('Apr', 'OIL3'): 125,
    ('May', 'VEG1'): 100,
    ('May', 'VEG2'): 120,
    ('May', 'OIL1'): 150,
    ('May', 'OIL2'): 110,
    ('May', 'OIL3'): 105,
    ('Jun', 'VEG1'): 90,
    ('Jun', 'VEG2'): 100,
    ('Jun', 'OIL1'): 140,
    ('Jun', 'OIL2'): 80,
    ('Jun', 'OIL3'): 135
}


hardness = {"VEG1": 8.8, "VEG2": 6.1, "OIL1": 2.0, "OIL2": 4.2, "OIL3": 5.0}

price = 150
init_store = 500
veg_cap = 200
oil_cap = 250

min_hardness = 3
max_hardness = 6
max_ingredients = 3
holding_cost = 5
min_consume = 20

Next, we create a model and the variables. We set the UpdateMode parameter to 1 (which simplifies the code – see the documentation for more details). For each period, we create a variable which will take into account the value of the food produced. For each product (five kinds of oils) and each period we will create variables for the amount that gets purchased, used, and stored.

For each period and each product, we need a binary variable, which indicates if this product is used in the current period.

In [2]:
food = Model('Food Manufacture II')
# Quantity of food produced in each period
produce = food.addVars(months, name="Food")
# Quantity bought of each product in each period
buy = food.addVars(months, oils, name = "Buy")
# Quantity used of each product  in each period
consume = food.addVars(months, oils, name = "Consume")
# Quantity stored of each product  in each period
store = food.addVars(months, oils, name = "Store")
# binary variables =1, if consume > 0
use = food.addVars(months, oils, vtype=GRB.BINARY, name = "Use")

Next, we insert the constraints. The balance constraints ensure that the amount of oil that is in the storage in the last period and the amount that gets purchased equals the amount that is used and stored for each oil in the current period. This ensures that all oil in the model was either in the initial storage or is purchased in some month.

In [3]:
#1. Initial Balance
food.addConstrs((init_store + buy[months[0], oil]
                 == consume[months[0], oil] + store[months[0], oil]
                 for oil in oils), "Initial_Balance")

#2.  Balance
food.addConstrs((store[months[months.index(month)-1], oil] + buy[month, oil]
                 == consume[month, oil] + store[month, oil]
                 for oil in oils for month in months if month != months[0]), "Balance")

{('VEG1', 'Feb'): <gurobi.Constr *Awaiting Model Update*>,
 ('VEG1', 'Mar'): <gurobi.Constr *Awaiting Model Update*>,
 ('VEG1', 'Apr'): <gurobi.Constr *Awaiting Model Update*>,
 ('VEG1', 'May'): <gurobi.Constr *Awaiting Model Update*>,
 ('VEG1', 'Jun'): <gurobi.Constr *Awaiting Model Update*>,
 ('VEG2', 'Feb'): <gurobi.Constr *Awaiting Model Update*>,
 ('VEG2', 'Mar'): <gurobi.Constr *Awaiting Model Update*>,
 ('VEG2', 'Apr'): <gurobi.Constr *Awaiting Model Update*>,
 ('VEG2', 'May'): <gurobi.Constr *Awaiting Model Update*>,
 ('VEG2', 'Jun'): <gurobi.Constr *Awaiting Model Update*>,
 ('OIL1', 'Feb'): <gurobi.Constr *Awaiting Model Update*>,
 ('OIL1', 'Mar'): <gurobi.Constr *Awaiting Model Update*>,
 ('OIL1', 'Apr'): <gurobi.Constr *Awaiting Model Update*>,
 ('OIL1', 'May'): <gurobi.Constr *Awaiting Model Update*>,
 ('OIL1', 'Jun'): <gurobi.Constr *Awaiting Model Update*>,
 ('OIL2', 'Feb'): <gurobi.Constr *Awaiting Model Update*>,
 ('OIL2', 'Mar'): <gurobi.Constr *Awaiting Model Update*

The end balance constraints force that at the end of the last period the storage contains the initial amount of each oil. The problem description demands that the storage is as full as in the beginning.

In [4]:
#3. Inventory Target
food.addConstrs((store[months[-1], oil] == init_store for oil in oils), "End_Balance")

{'VEG1': <gurobi.Constr *Awaiting Model Update*>,
 'VEG2': <gurobi.Constr *Awaiting Model Update*>,
 'OIL1': <gurobi.Constr *Awaiting Model Update*>,
 'OIL2': <gurobi.Constr *Awaiting Model Update*>,
 'OIL3': <gurobi.Constr *Awaiting Model Update*>}

The capacity constraints restrict the amount of veg and non-veg oils which can be processed per period. Per month only 200 tons of vegetable oil and 250 tons of non-vegetable oil can be processed to the final product.

In [5]:
#4.1 Vegetable Oil Capacity
food.addConstrs((quicksum(consume[month, oil] for oil in oils if "VEG" in oil)
                 <= veg_cap for month in months), "Capacity_Veg")

#4.2 Non-vegetable Oil Capacity
food.addConstrs((quicksum(consume[month, oil] for oil in oils if "OIL" in oil)
                 <= oil_cap for month in months), "Capacity_Oil")

{'Jan': <gurobi.Constr *Awaiting Model Update*>,
 'Feb': <gurobi.Constr *Awaiting Model Update*>,
 'Mar': <gurobi.Constr *Awaiting Model Update*>,
 'Apr': <gurobi.Constr *Awaiting Model Update*>,
 'May': <gurobi.Constr *Awaiting Model Update*>,
 'Jun': <gurobi.Constr *Awaiting Model Update*>}

The hardness constraints limit the hardness of the final product, which needs to remain between 3 and 6. Each oil has a certain hardness. The final product may be made up of different oils. The hardness of the final product is measured by the hardness of each ingredient multiplied by its share of the final product. It is assumed that the hardness blends linearly.

In [6]:
#5. Hardness
food.addConstrs((quicksum(hardness[oil]*consume[month, oil] for oil in oils)
                 >= min_hardness*produce[month] for month in months), "Hardness_lower")
food.addConstrs((quicksum(hardness[oil]*consume[month, oil] for oil in oils)
                 <= max_hardness*produce[month] for month in months), "Hardness_upper")

{'Jan': <gurobi.Constr *Awaiting Model Update*>,
 'Feb': <gurobi.Constr *Awaiting Model Update*>,
 'Mar': <gurobi.Constr *Awaiting Model Update*>,
 'Apr': <gurobi.Constr *Awaiting Model Update*>,
 'May': <gurobi.Constr *Awaiting Model Update*>,
 'Jun': <gurobi.Constr *Awaiting Model Update*>}

The conserve constraints ensure that the amount of products used in each period equals the amount of food produced in that period. This ensures that all oil that is used is also processed into the final product (food).

In [7]:
#6. Mass Conservation
food.addConstrs((consume.sum(month) == produce[month] for month in months), "Mass_conservation")

{'Jan': <gurobi.Constr *Awaiting Model Update*>,
 'Feb': <gurobi.Constr *Awaiting Model Update*>,
 'Mar': <gurobi.Constr *Awaiting Model Update*>,
 'Apr': <gurobi.Constr *Awaiting Model Update*>,
 'May': <gurobi.Constr *Awaiting Model Update*>,
 'Jun': <gurobi.Constr *Awaiting Model Update*>}

Condition 1 constraints force that if any product is used in any period then at least 20 tons is used. They also force that the binary variable for each product and each month is set to one if and only if the continuous variable used for the same product and the same month is non-zero. The binary variable is called an indicator variable since it is linked to a continuous variable and indicates if it is non-zero.

It's relatively straightforward to express Condition 1 as a pure MIP constraint set. Let's see how to model this set using Gurobi’s general constraints (from version 7.0 onwards):

In [8]:
#7.1 & 7.2 Consumption Range - Using Gurobi's General Constraints
for month in months:
    for oil in oils:
        food.addGenConstrIndicator(use[month, oil], 0,
                                   consume[month, oil] == 0,
                                   name="Lower_bound_{}_{}".format(month, oil))
        food.addGenConstrIndicator(use[month, oil], 1,
                                   consume[month, oil] >=  min_consume,
                                   name="Upper_bound_{}_{}".format(month, oil))

Condition 2 constraints ensure that each final product is only made up of at most three ingredients.

In [9]:
#8. Recipe
food.addConstrs((use.sum(month) <= max_ingredients for month in months),"Recipe")

{'Jan': <gurobi.Constr *Awaiting Model Update*>,
 'Feb': <gurobi.Constr *Awaiting Model Update*>,
 'Mar': <gurobi.Constr *Awaiting Model Update*>,
 'Apr': <gurobi.Constr *Awaiting Model Update*>,
 'May': <gurobi.Constr *Awaiting Model Update*>,
 'Jun': <gurobi.Constr *Awaiting Model Update*>}

Condition 3 constraints ensure that if vegetable one or vegetable two are used, then oil three must also be used. We will use again Gurobi's general constraints:

In [10]:
#9.1 & 9.2 If-then Constraint
for month in months:
    food.addGenConstrIndicator(use[month, "VEG1"], 1,
                               use[month, "OIL3"] == 1,
                               name = "If_then_a_{}".format(month))
    food.addGenConstrIndicator(use[month, "VEG2"], 1,
                               use[month, "OIL3"] == 1,
                               name = "If_then_b_{}".format(month))

The objective is to maximize the profit of the company. This is calculated as revenue minus costs for buying and storing of the purchased products (ingredients).

In [11]:
#0. Objective Function
obj = price*produce.sum() - buy.prod(cost) - holding_cost*store.sum()
food.setObjective(obj, GRB.MAXIMIZE) # maximize profit

Next, we start the optimization and Gurobi tries to find the optimal solution.

In [18]:
food.optimize()

Optimize a model with 71 rows, 126 columns and 288 nonzeros
Model has 72 general constraints
Variable types: 96 continuous, 30 integer (30 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [5e+00, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 5e+02]
Presolved: 113 rows, 71 columns, 322 nonzeros

Continuing optimization...


Cutting planes:
  Gomory: 9
  MIR: 1
  Flow cover: 2

Explored 530 nodes (2813 simplex iterations) in 0.01 seconds
Thread count was 8 (of 8 available processors)

Solution count 5: 100279 100192 99275 ... 18350

Optimal solution found (tolerance 1.00e-04)
Best objective 1.002787037037e+05, best bound 1.002787037037e+05, gap 0.0000%


---
## Analysis

When originally designed, this model proved comparatively hard to solve (see Food Manufacture I). The profit (revenue from sales minus cost of raw oils) resulting from this plan is 100,278.7 USD. There are alternative — and equally good — solutions.

### Purchase Plan

In [14]:
rows = months.copy()
columns = oils.copy()
purchase_plan = pd.DataFrame(columns=columns, index=rows, data=0.0)

for month, oil in buy.keys():
    if (abs(buy[month, oil].x) > 1e-6):
        purchase_plan.loc[month, oil] = np.round(buy[month, oil].x, 1)
purchase_plan

,VEG1,VEG2,OIL1,OIL2,OIL3
Jan,0.0,0.0,0.0,0.0,0.0
Feb,0.0,0.0,0.0,190.0,0.0
Mar,0.0,0.0,0.0,0.0,540.0
Apr,0.0,0.0,0.0,0.0,0.0
May,0.0,0.0,0.0,0.0,40.0
Jun,480.4,629.6,0.0,730.0,0.0


### Monthly Consumption

In [15]:
rows = months.copy()
columns = oils.copy()
reqs = pd.DataFrame(columns=columns, index=rows, data=0.0)

for month, oil in consume.keys():
    if (abs(consume[month, oil].x) > 1e-6):
        reqs.loc[month, oil] = np.round(consume[month, oil].x, 1)
reqs

,VEG1,VEG2,OIL1,OIL2,OIL3
Jan,0.0,200.0,0.0,230.0,20.0
Feb,85.2,114.8,0.0,0.0,250.0
Mar,85.2,114.8,0.0,0.0,250.0
Apr,155.0,0.0,0.0,230.0,20.0
May,155.0,0.0,0.0,230.0,20.0
Jun,0.0,200.0,0.0,230.0,20.0


### Inventory Plan

In [16]:
rows = months.copy()
columns = oils.copy()
store_plan = pd.DataFrame(columns=columns, index=rows, data=0.0)

for month, oil in store.keys():
    if (abs(store[month, oil].x) > 1e-6):
        store_plan.loc[month, oil] = np.round(store[month, oil].x, 1)
store_plan

,VEG1,VEG2,OIL1,OIL2,OIL3
Jan,500.0,300.0,500.0,270.0,480.0
Feb,414.8,185.2,500.0,460.0,230.0
Mar,329.6,70.4,500.0,460.0,520.0
Apr,174.6,70.4,500.0,230.0,500.0
May,19.6,70.4,500.0,0.0,520.0
Jun,500.0,500.0,500.0,500.0,500.0


Note: If you want to write your solution to a file, rather than print it to the terminal, you can use the model.write() command. An example implementation is:

`food.write("food-manufacture-2-output.sol")`

---
## References

H. Paul Williams, Model Building in Mathematical Programming, fifth edition (Pages 255, 298-300, 349-350)

Copyright © 2019 Gurobi Optimization, LLC